In [ ]:
import ipywidgets as widgets
from ipywidgets import interact
import matplotlib.pyplot as plt
import PIL, urllib
import numpy as np
import warnings
import pickle
warnings.filterwarnings("ignore", category=UserWarning) 

In [ ]:
%%javascript
document.title = "My SummerSchool App";

In [ ]:
default_url = 'https://user-images.githubusercontent.com/15421925/178372019-d765ca55-0a99-4420-a015-0dab9b05da13.png'

In [ ]:
display(widgets.HTML('''
<h1>
My Summer School App
</h1>
<p style="max-width: 500px;">
This app takes a url to a .jpg or .png image (white digit on a black background).
Converts it to a grey scale 8x8 image and passes
it through the model trained on a dataset of handwritten digits.
The predicted class is than simply displayed as a text.
</p>
<br>
<p>
Some example images from the test set:
<ul>
<li>https://user-images.githubusercontent.com/15421925/178372022-b7cd0ffc-4c8f-49d6-9cfb-8e7ff668b5c5.png</li>
<li>https://user-images.githubusercontent.com/15421925/178372024-6b717973-15b9-4e50-9cb3-1277579738e0.png</li>
<li>https://user-images.githubusercontent.com/15421925/178372026-16f259f7-b135-4acf-bf0c-f8f924ed09e5.png</li>
</ul>
<b>Type in an image url and press enter</b>.
</p>

'''))

In [ ]:
@interact(mt=widgets.RadioButtons(
    options=['LogisticRegression', 'PCANaiveBayes'],
    value='LogisticRegression',
    description='Model:',
    disabled=False
))
def f(mt):
    if mt == 'LogisticRegression':
        model_path = 'pretrained_lr.model'
    else:
        model_path = 'pretrained_nb.model'

    # Load the desired model
    model = pickle.load(open(model_path, 'rb'))

    @interact(
        url=widgets.Text(
            value=default_url,
            placeholder='Type in an image URL.',
            description='Image URL:',
            disabled=False,
            continuous_update=False))
    def g(url):
        try:
            # Load and transform the image
            img_arr = PIL.Image.open(
                urllib.request.urlopen(url)).convert('L').resize((8,8))
            
            # Keep the url change across model change
            global default_url
            default_url = url

            # Plot the transformed image
            plt.figure(figsize=(5,5))
            plt.imshow(img_arr, cmap='gray', aspect='equal')
            plt.title(f'Converted into grayscale {img_arr._size} image')
            plt.axis('off')
            plt.show()   

            # Preprocess the image data
            data = np.array(img_arr)
            data = np.round((data / data.max()) * 16)  # train data range (0 - 16)
            data = data.reshape(1, -1)  #
            
            # Pass it through the model and display the result
            display(f'Prediction: {model.predict(data)[0]}')
            
        except Exception as e:
            display(e)
